<a href="https://colab.research.google.com/github/Keltings/Accident-Severity-Analysis/blob/main/Accident_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. DATA OVERVIEW

```
`# This is formatted as code`
```

**

In [10]:
#import packages for data processing
import pandas as pd
import numpy as np

In [11]:
#In the first place we are going to import the dataset using pandas.
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/accidents 1.csv')

In [12]:
df.shape #check the number of rows and columns the dataset has

(867550, 33)

In [13]:
df.info() #show entry, datatype memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867550 entries, 0 to 867549
Data columns (total 33 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   Accident_Index                               867550 non-null  object 
 1   Location_Easting_OSGR                        867464 non-null  float64
 2   Location_Northing_OSGR                       867464 non-null  float64
 3   Longitude                                    867464 non-null  float64
 4   Latitude                                     867464 non-null  float64
 5   Police_Force                                 867550 non-null  int64  
 6   Accident_Severity                            867550 non-null  int64  
 7   Number_of_Vehicles                           867550 non-null  int64  
 8   Number_of_Casualties                         867550 non-null  int64  
 9   Date                                         867550 non-nul

The dataset has 867550 entries and 32 features + 1 target variable. Automatic Type Recognition: 13 of them are int type, 5 of them are float64, 15 of them are object. Memory usage: 218.4+ MB.



> **STATISTICS VIEW**



In [14]:
#Basic statistic on Ordinal, Interval and Ratio data.
OIR_columns = ['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties', 'Day_of_Week', 'Local_Authority_(District)', '1st_Road_Class', '1st_Road_Number', 'Speed_limit', '2nd_Road_Class', '2nd_Road_Number', 'Urban_or_Rural_Area', 'Year']
df[OIR_columns].describe() #Exclude Date; Time can be transformed to second/minute/hour/day/month/year separately, but it is no need to do it here.


,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Local_Authority_(District),1st_Road_Class,1st_Road_Number,Speed_limit,2nd_Road_Class,2nd_Road_Number,Urban_or_Rural_Area,Year
count,867464.000000,8.674640e+05,867464.000000,867464.000000,867550.000000,867550.000000,867550.000000,867550.000000,867550.000000,867550.000000,867550.000000,867550.000000,867550.000000,867550.000000,867550.000000,867550.000000,867550.000000
mean,439345.364558,2.993582e+05,-1.440835,52.582165,30.538545,2.841239,1.831130,1.355902,4.126450,350.888779,4.091393,1002.772014,39.169506,2.642198,381.527893,1.355084,2007.933220
std,95453.947190,1.613353e+05,1.403280,1.452735,25.637168,0.400021,0.715248,0.822754,1.926804,260.598261,1.431850,1822.998663,14.190679,3.205662,1305.163869,0.478821,2.326594
min,64950.000000,1.052000e+04,-7.516225,49.914488,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,10.000000,-1.000000,-1.000000,1.000000,2005.000000
25%,375220.000000,1.785300e+05,-2.371243,51.492749,7.000000,3.000000,1.000000,1.000000,2.000000,112.000000,3.000000,0.000000,30.000000,-1.000000,0.000000,1.000000,2006.000000
50%,439790.000000,2.663600e+05,-1.405855,52.288525,30.000000,3.000000,2.000000,1.000000,4.000000,323.000000,4.000000,124.000000,30.000000,3.000000,0.000000,1.000000,2009.000000
75%,523520.000000,3.969200e+05,-0.214757,53.467396,46.000000,3.000000,2.000000,1.000000,6.000000,531.000000,6.000000,709.000000,50.000000,6.000000,0.000000,2.000000,2010.000000
max,655370.000000,1.205100e+06,1.759398,60.724682,98.000000,3.000000,34.000000,87.000000,7.000000,941.000000,6.000000,9999.000000,70.000000,6.000000,9999.000000,3.000000,2011.000000


What can be derived from the above is that 2nd_Road_Class: 25%,50% and 75%are all zero which is quite abnormal
Junction_Detail: has null everything so should not be used for our analysis

In [15]:
#Basic statistics on the rest of the data

df.loc[:, ~df.columns.isin(OIR_columns)].astype("object").describe() # All the Nominal data can be treated as "object" type for simplicity.

,Accident_Index,Date,Time,Local_Authority_(Highway),Road_Type,Junction_Detail,Junction_Control,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
count,867550,867550,867466,867550,867550,0,516693,867533,867518,867550,867443,866546,867537,867524,865420,808937
unique,558085,2190,1439,207,6,0,4,3,6,5,9,5,8,6,2,34140
top,2010000000000,21/10/2005,17:00,E10000016,Single carriageway,NaN,Giveway or uncontrolled,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,None,Yes,E01000004
freq,137503,822,8531,22579,648718,NaN,419677,862339,726922,635472,695558,598491,845879,851446,699364,1749


Road_Type: Single carriageway seems to be a common road name that an accidents would happen. The name of the type of the road could mean something or contain some special characters.
Light_Conditions: It seems that accidents usually happen when on daylight when street lights are present.
Weather_Conditions: Fine without high winds  is the most frequent weather condition.

**Data Categorizing**

In [16]:
#rename the columns for a clear dataset and easy reading
accident_data = df.rename(columns={'Accident_Index': 'Index', 'Location_Easting_OSGR': 'LEO', 'Location_Northing_OSGR': 'LNO',
       'Longitude': 'Long', 'Latitude': 'Lat', 'Police_Force': 'PF', 'Accident_Severity': 'AS',
       'Number_of_Vehicles': 'NV', 'Number_of_Casualties': 'NC', 'Date': 'Date', 'Day_of_Week': 'DW',
       'Time': 'Time', 'Local_Authority_(District)': 'District', 'Local_Authority_(Highway)': 'Highway',
       '1st_Road_Class': 'RC1', '1st_Road_Number': 'RN1', 'Road_Type': 'RT', 'Speed_limit': 'SL',
       'Junction_Detail': 'JD', 'Junction_Control': 'JC', '2nd_Road_Class': 'RC2',
       '2nd_Road_Number': 'RN2', 'Pedestrian_Crossing-Human_Control': 'PCHC',
       'Pedestrian_Crossing-Physical_Facilities': 'PCPF', 'Light_Conditions': 'LC',
       'Weather_Conditions': 'WC', 'Road_Surface_Conditions': 'RSC',
       'Special_Conditions_at_Site': 'SCS', 'Carriageway_Hazards': 'CH',
       'Urban_or_Rural_Area': 'URA', 'Did_Police_Officer_Attend_Scene_of_Accident': 'DPOASA',
       'LSOA_of_Accident_Location': 'LSOAAL', 'Year': 'Year'})
accident_data.head(3)

,Index,LEO,LNO,Long,Lat,PF,AS,NV,NC,Date,DW,Time,District,Highway,RC1,RN1,RT,SL,JD,JC,RC2,RN2,PCHC,PCPF,LC,WC,RSC,SCS,CH,URA,DPOASA,LSOAAL,Year
0,200901BS70001,524910.0,180800.0,-0.201349,51.512273,1,2,2,1,1/1/2009,5,15:11,12,E09000020,6,0,One way street,30,NaN,Giveway or uncontrolled,6,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,E01002882,2009
1,200901BS70002,525050.0,181040.0,-0.199248,51.514399,1,2,2,11,5/1/2009,2,10:59,12,E09000020,5,0,Single carriageway,30,NaN,Giveway or uncontrolled,5,0,None within 50 metres,Zebra crossing,Daylight: Street light present,Fine without high winds,Wet/Damp,None,None,1,Yes,E01002886,2009
2,200901BS70003,526490.0,177990.0,-0.179599,51.486668,1,3,2,1,4/1/2009,1,14:19,12,E09000020,3,308,Single carriageway,30,NaN,Giveway or uncontrolled,6,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,E01002912,2009


In [17]:
accident_data.columns

Index(['Index', 'LEO', 'LNO', 'Long', 'Lat', 'PF', 'AS', 'NV', 'NC', 'Date',
       'DW', 'Time', 'District', 'Highway', 'RC1', 'RN1', 'RT', 'SL', 'JD',
       'JC', 'RC2', 'RN2', 'PCHC', 'PCPF', 'LC', 'WC', 'RSC', 'SCS', 'CH',
       'URA', 'DPOASA', 'LSOAAL', 'Year'],
      dtype='object')

In [18]:
#data visualization
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport
import plotly.express as px
import plotly.figure_factory as ff
import datetime as dt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

In [19]:
# Category Visualization
# https://plotly.com/python/sunburst-charts/
accident_data_category = dict(
    character=['Basic', 'Location', 'Environment', 'Infrastructure', 'Index', 'LEO', 'LNO', 'Long', 'Lat', 'PF', 'AS', 'NV', 'NC', 'Date',
               'DW', 'Time', 'District', 'Highway', 'RC1', 'RN1', 'RT', 'SL', 'JD',
               'JC', 'RC2', 'RN2', 'PCHC', 'PCPF', 'LC', 'WC', 'RSC', 'SCS', 'CH',
               'URA', 'DPOASA', 'LSOAAL', 'Year'],
     parent = ["","","","","Basic", "Location", "Location", "Location", "Location", "Basic", "Basic", "Basic",
               "Basic", "Basic", "Basic", "Basic", "Infrastructure", "Infrastructure", "Infrastructure", "Infrastructure", 
               "Infrastructure", "Infrastructure", "Infrastructure", "Infrastructure", "Infrastructure", "Infrastructure", 
               "Infrastructure", "Infrastructure", "Environment", "Environment", "Environment", "Environment", 
               "Environment", "Location", "Basic", "Basic", "Basic"],
)

category_fig =px.sunburst(
    accident_data_category,
    names='character',
    parents='parent'
)

category_fig.update_layout(
    autosize=False,
    width=1050,
    height=550,
    title={
        'text': "Data Categorizing",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
category_fig.show()

**2. DATA PREPROCESSING**

Here we perform the art of cleaning our data set  for better descriptive and predictive analysis
 

In [20]:
# Irrelevant columns
'''
INDEX: ID is unique and meaningless for the dataset.
LSOAAL: Not making sense.
Highway: doe not make sense
JD: The entire data is missing
'''
irrelavant_columns = ['Index','LSOAAL','Highway','JD']
accident_data1 = accident_data.drop(irrelavant_columns, axis=1)

In [21]:
#drop columns with missing values > 40%
# Replace the empty data with NaN
accident_data1.replace("", float("NaN"), inplace=True)
accident_data1.replace(" ", float("NaN"), inplace=True)

# Count missing value(NaN, na, null, None) of each columns, Then transform the result to a pandas dataframe. 
count_missing_value = accident_data1.isna().sum() / accident_data1.shape[0] * 100
count_missing_value_df = pd.DataFrame(count_missing_value.sort_values(ascending=False), columns=['Missing%'])

In [22]:
# Visualize the percentage(>0) of Missing value in each column.
missing_value_df = count_missing_value_df[count_missing_value_df['Missing%'] > 0]

plt.figure(figsize=(15, 5)) # Set the figure size
missing_value_graph = sns.barplot(y = missing_value_df.index, x = "Missing%", data=missing_value_df, orient="h")
missing_value_graph.set_title("Percentage Missing value of each feature", fontsize = 20)
missing_value_graph.set_ylabel("Features")

Text(0, 0.5, 'Features')

In [23]:
## Drop the column with Missing value(>40%)
missing_value_40_df = count_missing_value_df[count_missing_value_df['Missing%'] > 40]
accident_data1.drop(missing_value_40_df.index, axis=1, inplace=True)
missing_value_40_df

,Missing%
JC,40.44228


**Data Type Correcting**

In [24]:
# Convert Date, Time to datetime64[ns]
accident_data1['Date'] = pd.to_datetime(accident_data1['Date'])
accident_data1['Time'] = pd.to_datetime(accident_data1['Time'])

**Imputation:**
Chosing the suitable imputation tech which can highly represent the central tendency of the data.

In [25]:
# Display all the missing value
missing_value_df

,Missing%
JC,40.442280
DPOASA,0.245519
RSC,0.115728
WC,0.012334
LEO,0.009913
Lat,0.009913
LNO,0.009913
Long,0.009913
Time,0.009682
PCPF,0.003689


In [26]:
# Categorize the missing value to numerical and categorical for imputation purpose
numerical_missing = ['LEO', 'LNO', 'Lat', 'Long']
categorical_missing = ['DPOASA','RSC', 'WC', 'PCPF', 'PCHC', 'CH', 'SCS']

**Drop all NaN/NA/null**

In [27]:
# Drop all the instance with NaN/NA/null
accident_data1_dropNaN_df = accident_data1.dropna()
accident_data1_dropNaN_df.reset_index(drop=True, inplace=True)

**Median imputation**

In [28]:
# Imputation by corresponding class Median value 
accident_data1_median_df = accident_data1.copy()

# For numerical columns
for column_name in numerical_missing:
    accident_data1_median_df[column_name] = accident_data1_median_df.groupby('AS')[column_name].transform(lambda x:x.fillna(x.median()))

# # For categorical columns(Majority value imputation)
# https://medium.com/analytics-vidhya/best-way-to-impute-categorical-data-using-groupby-mean-mode-2dc5f5d4e12d
for column_name in categorical_missing:
    accident_data1_median_df[column_name] = accident_data1_median_df.groupby('AS')[column_name].transform(lambda x:x.fillna(x.fillna(x.mode().iloc[0])))

# Drop NaN and reset index
accident_data1_median_df.dropna(inplace=True)

**Mean imputation**

In [29]:
# Imputation by corresponding class Mean value 
accident_data1_mean_df = accident_data1.copy()

# For numerical columns
for column_name in numerical_missing:
    accident_data1_mean_df[column_name] = accident_data1_mean_df.groupby('AS')[column_name].transform(lambda x:x.fillna(x.mean()))

# For categorical columns(Majority value imputation)
for column_name in categorical_missing:
    accident_data1_mean_df[column_name] = accident_data1_mean_df.groupby('AS')[column_name].transform(lambda x:x.fillna(x.fillna(x.mode().iloc[0])))
    
# Drop NaN 
accident_data1_mean_df.dropna(inplace=True)
accident_data1_mean_df.shape

(867466, 28)

****After Data Imputation:****

accident_data1_dropNaN_df: The dataset that simply drop all the NaN values.
accident_data1_median_df: The dataset that use grouped median imputation for numerical data and grouped majority value imputation for categorical data.
accident_data1_mean_df: The dataset that use grouped mean imputation for numerical data and grouped majority value imputation for categorical data.




In [30]:
# Save these datasets to local
accident_data1_dropNaN_df.to_csv('accident_data1_dropNaN.csv', index=False)
accident_data1_median_df.to_csv('accident_data1_median.csv', index=False)
accident_data1_mean_df.to_csv('accident_data1_mean.csv', index=False)

In [31]:
# Choose the best dataset base on the performance of modeling
accident1_best_df = accident_data1_dropNaN_df.copy()
#accident1_best_df = data_preprocessed_median_df.copy()
#accident1_best_df = data_preprocessed_mean_df.copy()

# Reset index
accident1_best_df.reset_index(inplace=True)

***DATA ANALYSIS***


1.   Basic Analysis
2.   Infrasructure Analysis


1.   Environment Analysis
2.   Location Analysis





***Basic Analysis:***
What is the distribution of accidents severity (AS)?

In [32]:
# Count the number of each severity, transform the result to pandas dataframe
AS_counts = accident1_best_df["AS"].value_counts()
AS_counts_df = pd.DataFrame(AS_counts)

# Calculate the proportion of each Severity
AS_percentage_df = AS_counts_df / sum(AS_counts_df["AS"]) * 100

In [45]:
# Visualize the distribution of accidents severity
AS_fig = make_subplots(
    rows=1, cols=2, 
    specs=[[{"type": "xy"}, {"type": "domain"}]])

AS_fig.add_trace(go.Bar(x=AS_counts_df.index, 
                              y=AS_counts_df["AS"],
                              text=AS_counts_df["AS"],
                              textposition='outside',
                              showlegend=False),
                              1, 1)

AS_fig.add_trace(go.Pie(labels=AS_percentage_df.index, 
                     values=AS_percentage_df["AS"],
                     showlegend=True),
                     1, 2)

AS_fig.update_layout(
            height=600, 
            width=1000,
            title={
            'text': "The distribution of accidents severity",
            'font': {'size': 24},
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            )
AS_fig.update_xaxes(type='category')
AS_fig.show()

**As we can see from the graph, level 3 is the most frequent severity which is 85.4% of the total. That means our target variable(label) is quite unbalanced.**

What is the relaionship between number of vehicles NV and severity

In [79]:
# Calculate the no.ofvehicles of each Severity
no_veh = accident1_best_df.groupby('AS')["NV"].mean().round(2)
no_veh_df = pd.DataFrame(no_veh)

no_veh_fig = px.bar(no_veh_df, 
                        x = no_veh_df.index, 
                        y = 'NV', 
                        labels={"index": "AS"},
                        text = "NV")
no_veh_fig.update_layout(
    autosize=False,
    width=600,
    height=300,
    title={
        'text': "No of vehicles of each Severity",
        'y':0.95,
        'x':0.5,
        'font': {'size': 24},
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis={
        'type':'category'
    })
no_veh_fig.show()

Generally speaking, the number of vehicles has no much influence on the severity of the accident as seen on the above figure the average number of vehicle is more or less the same onthe type of severity

Wha t is the relationship between number of casualities NC and the Severity AS

In [84]:
# Calculate the no.ofvehicles of each Severity
no_veh = accident1_best_df.groupby('AS')["NC"].mean().round(2)
no_veh_df = pd.DataFrame(no_veh)

no_veh_fig = px.bar(no_veh_df, 
                        x = no_veh_df.index, 
                        y = 'NC', 
                        labels={"index": "AS"},
                        text = "NC")
no_veh_fig.update_layout(
    autosize=False,
    width=600,
    height=300,
    title={
        'text': "No of Casualities of each Severity",
        'y':0.95,
        'x':0.5,
        'font': {'size': 24},
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis={
        'type':'category'
    })
no_veh_fig.show()

*Location Analysis*

Is urban or rural area with the most accidents?

In [58]:
no_veh_df.columns

RangeIndex(start=0, stop=2, step=1)

In [50]:
accident1_best_df.columns

Index(['index', 'LEO', 'LNO', 'Long', 'Lat', 'PF', 'AS', 'NV', 'NC', 'Date',
       'DW', 'Time', 'District', 'RC1', 'RN1', 'RT', 'SL', 'RC2', 'RN2',
       'PCHC', 'PCPF', 'LC', 'WC', 'RSC', 'SCS', 'CH', 'URA', 'DPOASA',
       'Year'],
      dtype='object')

In [ ]:
accident_data.PCPF.value_counts()

In [ ]:
numerics = ['int64','float64']
numeric_df = df.select_dtypes(include=numerics)
len(numeric_df.columns)

The below codes were set up for pandas profiling which would ease our work of analysing the dataset

In [ ]:
#title Default title text
'''
data = df
data = ff.create_table(data.head(6))
data

!pip uninstall pandas_profiling

!pip install pandas-profiling[notebook,html]

profile = ProfileReport(df, title="accident analysis")
profile

profile.to_notebook_iframe()

profile.to_file(output_file="accident analysis.html")

profile.to_file(output_file="accident analysis.csv")

profile.to_file(output_file="your_report.json")
'''

Use of pandas profiling helps in analysing yur data more faster. Notice that we have missing values hence our data needs cleaning

In [ ]:
!pip install category_encoders

Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize

In [ ]:
import plotly.graph_objects as go
from nltk.corpus import stopwords

Exploratory Data Analysis (EDA)
Exploratory Data Analysis refers to the critical process of performing initial investigations on data so as to discover patterns,to spot anomalies,to test hypothesis and to check assumptions with the help of summary statistics and graphical representations. In this first part of this notebook we are going to do an EDA of the dataset.

In [ ]:
'''
print(accident_data["AS"].value_counts())
'''

In [ ]:
#accident_data.columns

In [ ]:
#accident_data1= accident_data
#accident_data1.shape

In [ ]:
'''accident_data1["Date"] = pd.to_datetime(accident_data1["Date"])

# Extract year, month, weekday and day
accident_data1["Year"] = accident_data1["Date"].dt.year
accident_data1["Month"] = accident_data1["Date"].dt.month
accident_data1["Weekday"] = accident_data1["Date"].dt.weekday
accident_data1["Day"] = accident_data1["Date"].dt.day

# Extract hour and minute
accident_data1['Time'] = pd.to_datetime(accident_data1['Time'])
accident_data1["Hour"] = accident_data1["Time"].dt.hour
accident_data1["Minute"] = accident_data1["Time"].dt.minute

accident_data.head(3)'''

In [ ]:
'''accident_data1 = accident_data1.drop(['Date', 'Time', 'JD', 'Weekday'], axis = 1)
accident_data1.shape'''


In [ ]:
'''# dropping duplicate values
accident_data1.drop_duplicates(keep=False,inplace=True)
accident_data1.shape'''